In [ ]:
%load_ext autoreload 

In [ ]:
%autoreload 2

In [ ]:
#!/usr/bin/env python
__author__ = "Kaspar Beelen"
import sys
sys.path.append('../')
from utils import utils_train,bias_utils
from gensim.models.word2vec import Word2Vec
import logging
import pandas as pd
from tqdm.notebook import tqdm
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [ ]:

# paths
ROOT = "/kbdata/ResearchDrive"
SENT_OUTPUT = "/kbdata/Processed/Sentences"
METADATA_PATH = "../../resources/Lijst_kranten_final.xlsx"

# model hyperparameter
SIZE = 300
WINDOW = 20
MIN_COUNT = 10
WORKERS = 8
EPOCH = 4
SEED = 42

# training data parameters
TRAIN_START = 1850
TRAIN_END = 1908
TRAIN_STEP = 10
TRAIN_WINDOW = 20

# contextual paramaters

FACETS = "Verspreidingsgebied"
#FACETS = "Provincie"
#FACETS = "Politek"
meta_df = pd.read_excel(METADATA_PATH,sheet_name="Sheet1",index_col=0)

In [ ]:
meta_df["PPN"] = meta_df["PPN"].astype(str)

In [ ]:
meta_df.head()

In [ ]:
list(meta_df.groupby(FACETS)['PPN'].apply(list).items())

In [ ]:



#for FACET,PPN in :
FACET,PPN =('Landelijk',
  ['832337900',
   '831178310',
   '832688045',
   '832675288',
   '832460079',
   '832667536',
   '830780084',
   '83308013X',
   '83138588X'])
#print(FACET, PPN)
#for START_YEAR in range(TRAIN_START,TRAIN_END,TRAIN_STEP):
START_YEAR = 1870; TRAIN_WINDOW = 10
print(f'Training model--from {START_YEAR} to {START_YEAR+TRAIN_WINDOW}--{FACET}');print(PPN);
        

In [ ]:
sentences = utils_train.SentIterator(ROOT,date_range=(START_YEAR,START_YEAR+TRAIN_WINDOW),
                                            processed_path=SENT_OUTPUT,
                                            ppn=PPN,tokenized=False,
                                            n_jobs=WORKERS)


In [ ]:
print('Initialing model and vocabulary')
model = Word2Vec(size=SIZE, window=WINDOW, min_count=MIN_COUNT, workers=WORKERS, seed=SEED)
model.build_vocab(sentences=sentences)
    
total_examples = model.corpus_count
print(f"\n-------\nTotal number examples = {total_examples}\n-------\n")

In [ ]:

#if not total_examples: continue
model.train(sentences=sentences, total_examples=total_examples, epochs=EPOCH)
MODEL_OUTPUT = "/kbdata/Processed/Models/{}-{}-{}.w2v.model".format(START_YEAR,START_YEAR + TRAIN_WINDOW,FACET.replace('/','_'))
model.save(MODEL_OUTPUT)
print(f'Saved model to {MODEL_OUTPUT}')


In [ ]:
import sys
sys.path.append('../')
from utils import utils_train,bias_utils
from gensim.models.word2vec import Word2Vec
import logging
import pandas as pd
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

# paths
ROOT = "/kbdata/ResearchDrive"
SENT_OUTPUT = "/kbdata/Processed/Sentences"

WORKERS = 8

# training parameters
START_YEAR = 1800
END_YEAR = 1909

#for START_YEAR,END_YEAR in range(TRAIN_STEP,TRAIN_END,TRAIN_STEP):
#for FACET,PPN in meta_df.groupby(FACET)['PPN'].apply(list).items(): 
print(f'Training model--from {START_YEAR} to {END_YEAR}')
for YEAR in range(START_YEAR,END_YEAR):
    sentences = utils_train.SentIterator(ROOT,date_range=(YEAR,YEAR),processed_path=SENT_OUTPUT,tokenized=False,n_jobs=WORKERS)
    print(sentences._date_range)
    sentences.prepareLines()
		


In [ ]:
def processZipParallel(file):
        """Iterate over files: select all files within a specific date range.
        Returns:
            a string separated by a <SEP> token.
        """
        # select relevant zip files
        selected = self._select_zip_by_date_range()
        print("Selected follow zip files = {selected}")
        self.count = 0
        for file in selected: 
            with ZipFile(file, 'r') as zipdata:
                
                # iterate over path to XML files in zip files
                # if the first sequenace of four integers, surrounded by '/' is in the set of included years (defined by _date_range)
                #  ## Need to check if this works
                article_text = [(zipdata.read(f),f) for f in zipdata.namelist() 
                                        if f.endswith(".xml") and int(re.findall(r"/([0-9]{4})/",'/' + f)[0]) 
                                            in set(self._date_range)]
                
                # read the XML files 
                article_text = Parallel(n_jobs=self._n_jobs)(delayed(read_doc)(zipdoc,f) for (zipdoc,f) in article_text)
                
                #print(len(article_text))
                # process documents
                for sent,doc_id in Parallel(n_jobs=self._n_jobs)(delayed(preprocess_sent)(at,doc_id,tokenized=False) 
                                    for at,doc_id in article_text):
                    self.count+=1
                    yield doc_id+"<SEP>"+sent

In [ ]:
!ls /kbdata/ResearchDrive/1820-1829.zip

In [ ]:
from zipfile import ZipFile

In [ ]:
zipdata.namelist()

In [ ]:
len(zipdata.namelist())

In [ ]:
import re

zipdata = ZipFile('/kbdata/ResearchDrive/1900-1909.zip','r')


In [ ]:
len(zipdata.namelist())

In [ ]:
[(f,int(re.findall(r"/(1[0-9]{3})/",'/' + f)[0])) for f in zipdata.namelist()[2845418:8945428]]